In [0]:
towers = spark.table('telecommunications.self_healing_networks.sf_mobilelocations')
display(towers)

In [0]:
from pyspark.sql import functions as F

current_status = (spark.table('telecommunications.self_healing_networks.device_status')
                  .sort('event_timestamp', ascending=False)
                  .dropDuplicates(['tower_id', 'device_id'])
                  .select(F.col('tower_id'), F.col('device'), F.col('status'))
                  .groupBy(F.col('tower_id'))
                  .agg(F.collect_list(F.col('status')).alias('device_status'))
                  .withColumn('Tower_Status', F.when(F.array_contains(F.col('device_status'), 'Telemetry Issue'), 
                         F.lit('Telemetry Issue Detected'))
                  .otherwise(F.lit('System Online and Functioning')))
                  .alias('status'))

# a_current_status = current_status.select(F.col('tower_id'), F.col('device'), F.col('status')).groupBy(F.col('tower_id'), F.col('device')).agg(F.first(F.col('status')).alias('status'))
spark.sql("drop table if exists telecommunications.self_healing_networks.current_status")
current_status = current_status.join(towers, on='tower_id', how='inner').select(F.col('tower_id'), F.col('Tower_Status'), F.col('lat'), F.col('lon'), F.col('range'))
display(current_status)
current_status.write.mode('overwrite').saveAsTable('telecommunications.self_healing_networks.current_status')


In [0]:
triaged = (spark.table('telecommunications.self_healing_networks.triaged_devices')
           .withColumn('solution_type', F.when(F.col('solution').contains('AUTOMATED FIX'), F.lit('Automated Fix'))
                       .otherwise(F.lit('Field Technician Deployed')))
           .withColumn('Fix Details', F.when(F.col('solution').contains('AUTOMATED FIX'), F.col('solution'))
                       .otherwise(F.lit('Field Technician Deployed')))
          .groupBy(F.col('tower_id'))
           .agg(F.map_from_arrays(F.collect_list(F.col('device')), F.collect_list(F.col('Fix Details'))).alias('device_fix_details'),
                # F.collect_list(F.col('status')).alias('status'),
                F.collect_list(F.col('solution_type')).alias('genai_status'))
)
display(triaged)

In [0]:

genai_status = current_status.join(triaged, on='tower_id', how='left').withColumn(
    'Overall_Fix_Status',
    F.when(F.col('device_fix_details').isNull(), F.lit('System Online and Functioning'))
     .when(F.array_contains(F.col('genai_status'), 'Automated Fix'), F.lit('Automated Fix'))
     .otherwise(F.lit('Field Technician Deployed'))
)
genai_status.write.mode('overwrite').saveAsTable('telecommunications.self_healing_networks.genai_status')